In [ ]:
#第一次請執行這裡安裝必要套件，第二次後就可以略過
!conda install pandas
!conda install matplotlib

In [ ]:
import datetime
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import dates as mdates
from matplotlib.dates import DateFormatter

In [ ]:
#讀取潮位資料
df = pd.read_csv("tide_TS_202301.csv")
df["Time"] = pd.to_datetime(df["Time"])
df

In [ ]:
#讀取影像拍攝時間
df_image = pd.read_csv("image_time_TS_202301.csv")
df_image["Time"] = pd.to_datetime(df_image["Time"])
df_image

In [ ]:
#內插影像拍攝時間點的潮位("Tag")作為y值
df["Tag"] = "0"
df_image["Tag"] = "1" #新增Tag欄位標記為:1，藉此跟潮位資料做區別
df_merge = pd.concat([df ,df_image], ignore_index=True)
df_merge = df_merge.sort_values(["Time"], ascending=True) #Time欄位必須為時間格式才能做排序
df_merge.set_index('Time', inplace=True)
df_merge["Tide"] = df_merge["Tide"].interpolate(method='time')
df_merge = df_merge.reset_index()
df_merge

In [ ]:
#設定雲量門檻
ct = 0.3

In [ ]:
x_tide = df_merge[(df_merge["Tag"] == "0")]["Time"]
y_tide = df_merge[(df_merge["Tag"] == "0")]["Tide"]
x_imagetime_l = df_merge[(df_merge["Tag"] == "1")&(df_merge["Cloud"] < ct)]["Time"]
y_imagetime_l = df_merge[(df_merge["Tag"] == "1")&(df_merge["Cloud"] < ct)]["Tide"]
x_imagetime_h = df_merge[(df_merge["Tag"] == "1")&(df_merge["Cloud"] >= ct)]["Time"]
y_imagetime_h = df_merge[(df_merge["Tag"] == "1")&(df_merge["Cloud"] >= ct)]["Tide"]

fig = plt.figure(figsize=(20,7))
plt.rcParams["font.size"] = 16
ax = plt.subplot(111)

ax.plot(x_tide, y_tide, linewidth=1)
ax.plot(x_imagetime_l, y_imagetime_l, 'ro')
ax.plot(x_imagetime_h, y_imagetime_h, 'bo')

ax.set_title("Image capture and Tide")
ax.set_xlabel("Time")
ax.set_ylabel("Tide Level(cm)")

#ax.minorticks_on()
ax.tick_params(axis="both", which="major",direction="in",length=7,width=2,top="on",right="on")
ax.tick_params(axis="both", which="minor",direction="in",length=4,width=1,top="on",right="on")
ax.xaxis.set_major_formatter(DateFormatter("%Y-%m-%d"))
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
#ax.xaxis.set_minor_locator(mdates.MonthLocator(interval=1))

ax.set_ylim(y_tide.min()*1.2, y_tide.max()*1.2)
ax.set_xlim(datetime.datetime(2022, 11, 1), datetime.datetime(2023, 1, 10))

plt.show()